In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 200)

In [2]:
donnees_brutes = pd.read_csv('../Data/donnees_propres.csv')
donnees_brutes

,Unnamed: 0,Région,Municipalite,Quartier,Type_propriete,Nombre_ventes,Volume_ventes,Delai,Prix_vente_moyen,Prix_vs_inscription,Prix_vs_evaluation,Géographie
0,0,Abitibi-Témiscamingue,Clermont,Aucun,Total_proprietes,0,0.0,0,NaN,NaN,NaN,Municipalité
1,1,Abitibi-Témiscamingue,Clermont,Aucun,Unifamiliale,0,0.0,0,0.0,0.0,0.0,Municipalité
2,2,Abitibi-Témiscamingue,Clermont,Aucun,Coprop./App.,0,0.0,0,0.0,0.0,0.0,Municipalité
3,3,Abitibi-Témiscamingue,Clermont,Aucun,Prop. à revenus,0,0.0,0,NaN,NaN,NaN,Municipalité
4,4,Abitibi-Témiscamingue,Clermont,Aucun,Ferme/Fermette,0,0.0,0,NaN,NaN,NaN,Municipalité
...,...,...,...,...,...,...,...,...,...,...,...,...
13354,13081,Saguenay–Lac-Saint-Jean,Saguenay (Shipshaw),Aucun,Coprop./App.,0,0.0,0,0.0,0.0,0.0,Municipalité
13355,13082,Saguenay–Lac-Saint-Jean,Saguenay (Shipshaw),Aucun,Prop. à revenus,0,0.0,0,NaN,NaN,NaN,Municipalité
13356,13083,Saguenay–Lac-Saint-Jean,Saguenay (Shipshaw),Aucun,Ferme/Fermette,0,0.0,0,NaN,NaN,NaN,Municipalité
13357,13084,Saguenay–Lac-Saint-Jean,Saguenay (Shipshaw),Aucun,Terre/Terrain,2,301200.0,16,NaN,NaN,NaN,Municipalité


## Réunir les données résidentielles (avec quartiers et municipalités)

In [3]:
residentiel = donnees_brutes[(donnees_brutes['Type_propriete'] == 'Unifamiliale') | (donnees_brutes['Type_propriete'] == 'Coprop./App.')]
residentiel['ventes*prix_moyen'] = residentiel['Nombre_ventes'] * residentiel['Prix_vente_moyen']
residentiel['ventes*inscript'] = residentiel['Nombre_ventes'] * residentiel['Prix_vs_inscription']
residentiel['ventes*eval'] = residentiel['Nombre_ventes'] * residentiel['Prix_vs_evaluation']
residentiel['ventes*delais'] = residentiel['Nombre_ventes'] * residentiel['Delai']

<ipython-input-3-c11743bbdc28>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residentiel['ventes*prix_moyen'] = residentiel['Nombre_ventes'] * residentiel['Prix_vente_moyen']
<ipython-input-3-c11743bbdc28>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residentiel['ventes*inscript'] = residentiel['Nombre_ventes'] * residentiel['Prix_vs_inscription']
<ipython-input-3-c11743bbdc28>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [4]:
residentiel_pivot = pd.pivot_table(residentiel, index=['Région', 'Municipalite', 'Quartier'], values=['Nombre_ventes', 'ventes*prix_moyen', 'ventes*inscript', 'ventes*eval', 'ventes*delais'], aggfunc='sum').reset_index()
residentiel_pivot['prix_moyen'] = round(residentiel_pivot['ventes*prix_moyen'] / residentiel_pivot['Nombre_ventes'])
residentiel_pivot['diff_inscription'] = round(residentiel_pivot['ventes*inscript'] / residentiel_pivot['Nombre_ventes'])
residentiel_pivot['diff_evaluation'] = round(residentiel_pivot['ventes*eval'] / residentiel_pivot['Nombre_ventes'])
residentiel_pivot['delais_moyens'] = round(residentiel_pivot['ventes*delais'] / residentiel_pivot['Nombre_ventes'])

residentiel_pivot = residentiel_pivot.sort_values(by='prix_moyen').drop(columns=['ventes*eval', 'ventes*inscript', 'ventes*prix_moyen', 'ventes*delais'])
residentiel_pivot[['delais_moyens', 'Nombre_ventes', 'prix_moyen', 'diff_inscription', 'diff_evaluation']] = residentiel_pivot[['delais_moyens', 'Nombre_ventes', 'prix_moyen', 'diff_inscription', 'diff_evaluation']].fillna(0).astype(int)
residentiel_pivot = residentiel_pivot.reset_index().drop(columns='index')
residentiel_pivot

,Région,Municipalite,Quartier,Nombre_ventes,prix_moyen,diff_inscription,diff_evaluation,delais_moyens
0,Saguenay–Lac-Saint-Jean,Mont-Valin,Mont-Valin,1,0,0,0,231
1,Côte-Nord,Chute-aux-Outardes,Aucun,1,0,0,0,450
2,Mauricie,Rivière-de-la-Savane,Aucun,1,0,0,0,30
3,Chaudière-Appalaches,Saint-Cyprien,Aucun,1,35000,70,117,32
4,Mauricie,Sainte-Angèle-de-Prémont,Aucun,1,39000,71,134,131
...,...,...,...,...,...,...,...,...
1869,Saguenay–Lac-Saint-Jean,Saint-Thomas-Didyme,Aucun,0,0,0,0,0
1870,Saguenay–Lac-Saint-Jean,Sainte-Hedwidge,Aucun,0,0,0,0,0
1871,Saguenay–Lac-Saint-Jean,Sainte-Hedwidge,Sainte-Hedwidge,0,0,0,0,0
1872,Saguenay–Lac-Saint-Jean,Sainte-Hedwidge,Zec de la Lièvre,0,0,0,0,0


In [5]:
residentiel_pivot.to_csv('../Data/donnees_quartiers.csv')

In [21]:
test_surenchere = residentiel_pivot
test_surenchere['surenchere'] = round(test_surenchere['prix_moyen'] * (test_surenchere['diff_inscription']/100) - test_surenchere['prix_moyen'], 1)
test_surenchere[test_surenchere['Nombre_ventes'] > 2].sort_values(by='delais_moyens', ascending=False)

,Région,Municipalite,Quartier,Nombre_ventes,prix_moyen,diff_inscription,diff_evaluation,delais_moyens,surenchere
127,Capitale-Nationale,Saint-Siméon,Aucun,5,134800,92,111,318,-10784.0
499,Bas-Saint-Laurent,Sainte-Luce,Luceville,4,254750,91,157,316,-22927.5
802,Capitale-Nationale,Baie-Saint-Paul,Aucun,13,342308,99,125,287,-3423.1
507,Abitibi-Témiscamingue,Rivière-Héva,Aucun,3,257667,98,104,274,-5153.3
810,Chaudière-Appalaches,Saint-Raphaël,Aucun,3,344667,89,119,263,-37913.4
...,...,...,...,...,...,...,...,...,...
713,Outaouais,La Pêche,Lac-des-Loups,3,313333,120,167,6,62666.6
1499,Montréal,Beaconsfield,Sud Est,3,827500,119,177,6,157225.0
1528,Montérégie,Saint-Lazare,Dynastie/Sandbrook,3,1024333,99,162,6,-10243.3
466,Estrie,Frontenac,Aucun,3,245167,101,134,5,2451.7


In [7]:
test_surenchere.to_csv('../Data/donnees_avec_surencheres.csv')

## Réunir les données résidentielles (municipalités seulement)

In [8]:
residentiel_municipalites = donnees_brutes[((donnees_brutes['Type_propriete'] == 'Unifamiliale') | (donnees_brutes['Type_propriete'] == 'Coprop./App.')) & (donnees_brutes['Quartier'] == 'Aucun')]
residentiel_municipalites

,Unnamed: 0,Région,Municipalite,Quartier,Type_propriete,Nombre_ventes,Volume_ventes,Delai,Prix_vente_moyen,Prix_vs_inscription,Prix_vs_evaluation,Géographie
1,1,Abitibi-Témiscamingue,Clermont,Aucun,Unifamiliale,0,0.0,0,0.0,0.0,0.0,Municipalité
2,2,Abitibi-Témiscamingue,Clermont,Aucun,Coprop./App.,0,0.0,0,0.0,0.0,0.0,Municipalité
8,8,Abitibi-Témiscamingue,Rivière-Ojima,Aucun,Unifamiliale,0,0.0,0,0.0,0.0,0.0,Municipalité
9,9,Abitibi-Témiscamingue,Rivière-Ojima,Aucun,Coprop./App.,0,0.0,0,0.0,0.0,0.0,Municipalité
15,33,Abitibi-Témiscamingue,Saint-Lambert,Aucun,Unifamiliale,0,0.0,0,0.0,0.0,0.0,Municipalité
...,...,...,...,...,...,...,...,...,...,...,...,...
13326,13053,Saguenay–Lac-Saint-Jean,Saguenay (Lac-Kénogami),Aucun,Coprop./App.,0,0.0,0,0.0,0.0,0.0,Municipalité
13332,13059,Saguenay–Lac-Saint-Jean,Saguenay (Laterrière),Aucun,Unifamiliale,11,3310500.0,27,300955.0,101.0,127.0,Municipalité
13333,13060,Saguenay–Lac-Saint-Jean,Saguenay (Laterrière),Aucun,Coprop./App.,5,962500.0,36,192500.0,100.0,116.0,Municipalité
13353,13080,Saguenay–Lac-Saint-Jean,Saguenay (Shipshaw),Aucun,Unifamiliale,9,2131000.0,44,236778.0,95.0,114.0,Municipalité


In [9]:
residentiel_municipalites['ventes*prix_moyen'] = residentiel_municipalites['Nombre_ventes'] * residentiel_municipalites['Prix_vente_moyen']
residentiel_municipalites['ventes*inscript'] = residentiel_municipalites['Nombre_ventes'] * residentiel_municipalites['Prix_vs_inscription']
residentiel_municipalites['ventes*eval'] = residentiel_municipalites['Nombre_ventes'] * residentiel_municipalites['Prix_vs_evaluation']
residentiel_municipalites['ventes*delais'] = residentiel_municipalites['Nombre_ventes'] * residentiel_municipalites['Delai']

<ipython-input-9-0dba4dbe67b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residentiel_municipalites['ventes*prix_moyen'] = residentiel_municipalites['Nombre_ventes'] * residentiel_municipalites['Prix_vente_moyen']
<ipython-input-9-0dba4dbe67b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residentiel_municipalites['ventes*inscript'] = residentiel_municipalites['Nombre_ventes'] * residentiel_municipalites['Prix_vs_inscription']
<ipython-input-9-0dba4dbe67b8>:3: SettingWithCopyWarning: 
A value is t

In [10]:
residentiel_municipalites_pivot = pd.pivot_table(residentiel_municipalites, index=['Région', 'Municipalite', 'Quartier'], values=['Nombre_ventes', 'ventes*prix_moyen', 'ventes*inscript', 'ventes*eval', 'ventes*delais'], aggfunc='sum').reset_index()
residentiel_municipalites_pivot['prix_moyen'] = round(residentiel_municipalites_pivot['ventes*prix_moyen'] / residentiel_municipalites_pivot['Nombre_ventes'])
residentiel_municipalites_pivot['diff_inscription'] = round(residentiel_municipalites_pivot['ventes*inscript'] / residentiel_municipalites_pivot['Nombre_ventes'])
residentiel_municipalites_pivot['diff_evaluation'] = round(residentiel_municipalites_pivot['ventes*eval'] / residentiel_municipalites_pivot['Nombre_ventes'])
residentiel_municipalites_pivot['delais_moyens'] = round(residentiel_municipalites_pivot['ventes*delais'] / residentiel_municipalites_pivot['Nombre_ventes'])

residentiel_municipalites_pivot = residentiel_municipalites_pivot.sort_values(by='prix_moyen').drop(columns=['ventes*eval', 'ventes*inscript', 'ventes*prix_moyen', 'ventes*delais'])
residentiel_municipalites_pivot[['prix_moyen', 'diff_inscription', 'diff_evaluation', 'delais_moyens']] = residentiel_municipalites_pivot[['prix_moyen', 'diff_inscription', 'diff_evaluation', 'delais_moyens']].fillna(0).astype(int)
residentiel_municipalites_pivot = residentiel_municipalites_pivot.reset_index().drop(columns=['index', 'Quartier'])

### ==> Faire matcher les municipalités à celles du MAPH

In [11]:
equiv_maph = pd.read_csv('../Data/equivalences_centris_maph.csv')
munis_maph = pd.read_csv('../Data/match_villes_maph.csv')
print(equiv_maph.shape)
print(munis_maph.shape)

(71, 2)
(1153, 6)


In [12]:
municipalites = pd.merge(munis_maph, equiv_maph, how='outer', on='Municipalité MAPH')
municipalites['Muni_Centris'] = np.where(municipalites['Municipalité Centris_y'].isnull(), municipalites['Municipalité Centris_x'], municipalites['Municipalité Centris_y'])
municipalites = municipalites.drop(columns=['Municipalité Centris_x', 'Municipalité Centris_y'])
municipalites

,Code,Région,MRC \ CM,Municipalité MAPH,Désignation,Muni_Centris
0,46005.0,Estrie,Brome-Missisquoi,Abercorn,Village,Abercorn
1,48028.0,Montérégie,Acton,Acton Vale,Ville,Acton Vale
2,31056.0,Chaudière-Appalaches,Les Appalaches,Adstock,Municipalité,Adstock
3,98030.0,Côte-Nord,Minganie,Aguanish,Municipalité,Aguanish
4,92030.0,Saguenay–Lac-Saint-Jean,Maria-Chapdelaine,Albanel,Municipalité,Albanel
...,...,...,...,...,...,...
1208,49040.0,Centre-du-Québec,Drummond,Wickham,Municipalité,Wickham
1209,42088.0,Estrie,Le Val-Saint-François,Windsor,Ville,Windsor
1210,40017.0,Estrie,Les Sources,Wotton,Municipalité,Wotton
1211,51020.0,Mauricie,Maskinongé,Yamachiche,Municipalité,Yamachiche


In [13]:
municipalites_merged = pd.merge(residentiel_municipalites_pivot, municipalites, left_on=['Région', 'Municipalite'], right_on=['Région', 'Muni_Centris'], how='left')

municipalites_merged['ventes*prix_moyen'] = municipalites_merged['Nombre_ventes'] * municipalites_merged['prix_moyen']
municipalites_merged['ventes*inscript'] = municipalites_merged['Nombre_ventes'] * municipalites_merged['diff_inscription']
municipalites_merged['ventes*eval'] = municipalites_merged['Nombre_ventes'] * municipalites_merged['diff_evaluation']
municipalites_merged['delais_moyens*ventes'] = municipalites_merged['Nombre_ventes'] * municipalites_merged['delais_moyens']

In [14]:
municipalites_pivot = pd.pivot_table(municipalites_merged, index=['Région', 'MRC \ CM', 'Municipalite', 'Municipalité MAPH', 'Code'], values=['Nombre_ventes', 'ventes*prix_moyen', 'ventes*inscript', 'ventes*eval', 'delais_moyens*ventes'], aggfunc='sum')

municipalites_pivot['prix_moyen2'] = round(municipalites_pivot['ventes*prix_moyen'] / municipalites_pivot['Nombre_ventes'])
municipalites_pivot['diff_inscription2'] = round(municipalites_pivot['ventes*inscript'] / municipalites_pivot['Nombre_ventes'])
municipalites_pivot['diff_evaluation2'] = round(municipalites_pivot['ventes*eval'] / municipalites_pivot['Nombre_ventes'])
municipalites_pivot['delais_moyens2'] = round(municipalites_pivot['delais_moyens*ventes'] / municipalites_pivot['Nombre_ventes'])

municipalites_pivot = municipalites_pivot.sort_values(by='prix_moyen2').drop(columns=['ventes*eval', 'ventes*inscript', 'ventes*prix_moyen', 'delais_moyens*ventes'])
municipalites_pivot[['Nombre_ventes', 'prix_moyen2', 'diff_inscription2', 'diff_evaluation2', 'delais_moyens2']] = municipalites_pivot[['Nombre_ventes', 'prix_moyen2', 'diff_inscription2', 'diff_evaluation2', 'delais_moyens2']].fillna(0).astype(int)
municipalites_pivot = municipalites_pivot.reset_index()

In [15]:
municipalites_pivot.to_csv('../Data/donnees_municipalites.csv')

### Réunir par région

In [16]:
municipalites_merged.columns

Index(['Région', 'Municipalite', 'Nombre_ventes', 'prix_moyen',
       'diff_inscription', 'diff_evaluation', 'delais_moyens', 'Code',
       'MRC \ CM', 'Municipalité MAPH', 'Désignation', 'Muni_Centris',
       'ventes*prix_moyen', 'ventes*inscript', 'ventes*eval',
       'delais_moyens*ventes'],
      dtype='object')

In [17]:
residentiel_region_pivot = pd.pivot_table(municipalites_merged, index=['Région'], values=['Nombre_ventes', 'ventes*prix_moyen', 'ventes*inscript', 'ventes*eval', 'delais_moyens*ventes'], aggfunc='sum').reset_index()
residentiel_region_pivot['prix_moyen'] = round(residentiel_region_pivot['ventes*prix_moyen'] / residentiel_region_pivot['Nombre_ventes'])
residentiel_region_pivot['diff_inscription'] = round(residentiel_region_pivot['ventes*inscript'] / residentiel_region_pivot['Nombre_ventes'])
residentiel_region_pivot['diff_evaluation'] = round(residentiel_region_pivot['ventes*eval'] / residentiel_region_pivot['Nombre_ventes'])
residentiel_region_pivot['delais_moyens'] = round(residentiel_region_pivot['delais_moyens*ventes'] / residentiel_region_pivot['Nombre_ventes'])

residentiel_region_pivot['surenchere'] = round(residentiel_region_pivot['prix_moyen'] * (residentiel_region_pivot['diff_inscription']/100) - residentiel_region_pivot['prix_moyen'], 1)

residentiel_region_pivot = residentiel_region_pivot.sort_values(by='prix_moyen').drop(columns=['ventes*eval', 'ventes*inscript', 'ventes*prix_moyen', 'delais_moyens*ventes'])
residentiel_region_pivot[['prix_moyen', 'diff_inscription', 'diff_evaluation', 'delais_moyens']] = residentiel_region_pivot[['prix_moyen', 'diff_inscription', 'diff_evaluation', 'delais_moyens']].fillna(0).astype(int)
residentiel_region_pivot = residentiel_region_pivot.reset_index().drop(columns=['index'])
residentiel_region_pivot.sort_values(by='surenchere')

,Région,Nombre_ventes,prix_moyen,diff_inscription,diff_evaluation,delais_moyens,surenchere
0,Côte-Nord,154,175086,94,108,100,-10505.2
7,Abitibi-Témiscamingue,288,274322,97,130,84,-8229.7
1,Nord-du-Québec,30,196113,96,110,126,-7844.5
4,Saguenay–Lac-Saint-Jean,538,233248,97,124,63,-6997.4
3,Gaspésie–Îles-de-la-Madeleine,120,229273,97,142,68,-6878.2
2,Bas-Saint-Laurent,472,215461,97,134,87,-6463.8
8,Centre-du-Québec,548,295645,100,149,54,0.0
5,Chaudière-Appalaches,1144,269512,100,127,71,0.0
9,Capitale-Nationale,2514,328800,101,123,71,3288.0
6,Mauricie,589,269640,104,147,41,10785.6


In [18]:
residentiel_region_pivot.to_csv(('../Data/donnees_regions.csv'))

### Faire des tests

In [19]:
# Voir quels prix moyens inscrits par Centris sont loin du prix total / nombre de ventes --> vérifié dans le document original et c'est bien ça

test_moyennes = residentiel
test_moyennes['prix_moyen_calcule'] = round(test_moyennes['Volume_ventes'] / test_moyennes['Nombre_ventes'])
test_moyennes['diff_prix'] = test_moyennes['prix_moyen_calcule'] - test_moyennes['Prix_vente_moyen']
test_moyennes[test_moyennes['Nombre_ventes'] > 2][['Région', 'Municipalite', 'Quartier', 'Type_propriete', 'Prix_vente_moyen', 'diff_prix']].sort_values(by='diff_prix')

<ipython-input-19-f5527424dc89>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_moyennes['prix_moyen_calcule'] = round(test_moyennes['Volume_ventes'] / test_moyennes['Nombre_ventes'])
<ipython-input-19-f5527424dc89>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_moyennes['diff_prix'] = test_moyennes['prix_moyen_calcule'] - test_moyennes['Prix_vente_moyen']


,Région,Municipalite,Quartier,Type_propriete,Prix_vente_moyen,diff_prix
10828,Montréal,Beaconsfield,Sud Est,Unifamiliale,827500.0,-275833.0
10361,Montérégie,Saint-Mathieu,Aucun,Unifamiliale,436500.0,-148257.0
3942,Chaudière-Appalaches,Beaulac-Garthby,Aucun,Unifamiliale,529000.0,-144333.0
9233,Montérégie,McMasterville,Aucun,Coprop./App.,328500.0,-109500.0
8868,Montérégie,Beauharnois,Maple Grove,Unifamiliale,717143.0,-103143.0
...,...,...,...,...,...,...
6597,Lanaudière,L'Assomption,Aucun,Coprop./App.,329858.0,0.0
13353,Saguenay–Lac-Saint-Jean,Saguenay (Shipshaw),Aucun,Unifamiliale,236778.0,0.0
4836,Côte-Nord,Pointe-Lebel,Aucun,Unifamiliale,105250.0,3875.0
12942,Saguenay–Lac-Saint-Jean,Ferland-et-Boilleau,Aucun,Unifamiliale,279852.0,59059.0
